In [54]:
import requests

In [55]:
from bs4 import BeautifulSoup
import geopandas as gpd
import numpy as np
import pandas as pd

## Open GIS Data

In [56]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Wine Data

In [57]:
r = requests.get(
    "https://worldpopulationreview.com/state-rankings/wine-production-by-state"
)

In [58]:
soup = BeautifulSoup(r.text, "html.parser")

In [59]:
states = []
amounts = []
for tr in soup.find_all("tr", {"class": "table-row"}):
    state = tr.find_all("th")[0].text
    amount = tr.find("td").text
    if "K" in amount:
        amount = int(float(amount.split("K")[0]) * 1000)
    elif "M" in amount:
        amount = int(float(amount.split("M")[0]) * 1000000)
    else:
        continue
    states.append(state)
    amounts.append(amount)

In [63]:
wine_df = pd.DataFrame({"NAME": states, "WINE_G": amounts})
wine_df = wine_df.drop_duplicates()

## Merge Data

In [71]:
wine_per_state_df = states_df[["geometry", "NAME"]].merge(
    wine_df, on="NAME", how="left"
)

In [72]:
wine_per_state_df = wine_per_state_df.fillna(0)
wine_per_state_df["WINE_G"] = wine_per_state_df["WINE_G"].astype(int)

In [73]:
wine_per_state_df = wine_per_state_df.to_crs(9311)

In [74]:
wine_per_state_df["g_per_km"] = wine_per_state_df["WINE_G"] / (
    wine_per_state_df["geometry"].area / 1_000_000
)
wine_per_state_df["km_per_g"] = (
    wine_per_state_df["geometry"].area / 1_000_000
) / wine_per_state_df["WINE_G"].astype(float)

wine_per_state_df["bottles"] = wine_per_state_df["WINE_G"].astype(float) * 5.0472

wine_per_state_df["bottles_per_km"] = wine_per_state_df["bottles"] / (
    wine_per_state_df["geometry"].area / 1_000_000
)
wine_per_state_df["km_per_bottles"] = (
    wine_per_state_df["geometry"].area / 1_000_000
) / wine_per_state_df["bottles"]

wine_per_state_df = wine_per_state_df.fillna(0)
wine_per_state_df = wine_per_state_df.replace(to_replace=[np.inf, -np.inf], value=0)

In [75]:
wine_per_state_df.to_file("data/wine_per_state.gpkg")